In [3]:
from gurobipy import *

In [2]:
!pip install gurobipy

You should consider upgrading via the 'C:\Users\daiki\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [ ]:
#Sets
Cakes = ["Chocolate","Pound"]
Ingredients = ["Time","Eggs","Milk"]

C = range(len(Cakes))
I = range(len(Ingredients))

#Data
price = [400, 200]
available = [8*60, 30, 5]
usage = [
        [20,50],
        [4,1],
        [0.25,0.2]
        ]

m = Model("IP")

#decide variables
X = {}
for c in C:
    X[c] = m.addVar(vtype=GRB.INTEGER) #set integer

#objective function
m.setObjective(quicksum(price[c]*X[c] for c in C), GRB.MAXIMIZE)

#constraints
for i in I:
    m.addConstr(quicksum(usage[i][c]*X[c] for c in C) <= available[i])

#solver
m.optimize()

#result
for c in C:
    print("Bake", X[c].x, Cakes[c])
print("Revenue is", m.objVal)

In [118]:
S=[i for i in range(1,5)]
H=[i for i in range(1,4)]
P=[i for i in range(1,9)]
T=[i for i in range(1,5)]
n=[1 for i in range(1,9)]


In [119]:
m=Model("IP3")

In [120]:
x,y={},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")

for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")



In [121]:
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

In [127]:
for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])

In [128]:
for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      m.addConstr(quicksum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))<=quicksum(x[s,h,k,len(T)] for k in range(p,len(P)+1)))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[s,k,t+1] for s in S if s!=k)<=1)

In [129]:
m.addConstr(x[4,1,1,1]==1)
m.addConstr(x[4,2,2,1]==1)
m.addConstr(x[2,1,3,1]==1)
m.addConstr(x[1,2,4,1]==1)
m.addConstr(x[1,1,5,1]==1)
m.addConstr(x[3,2,6,1]==1)
m.addConstr(x[3,3,7,1]==1)
m.addConstr(x[3,1,8,1]==1)


<gurobi.Constr *Awaiting Model Update*>

In [130]:
m.setObjective(quicksum(y[s,k,t] for s in S for k in S if k!=s for t in range(1,len(T))))
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1607 rows, 420 columns and 10244 nonzeros
Model fingerprint: 0xed690fe1
Variable types: 0 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R903 by 1.000000000

Presolve removed 1607 rows and 420 columns
Presolve time: 0.07s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.11 seconds (0.02 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [131]:
EPS=1.e-6

for (s,h,p,t) in x:
  if t==1:
    if x[s,h,p,t].X>EPS:
      print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))
  if t>=2:
    if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
      print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

for (s,k,t) in y:
  if y[s,k,t].X>EPS:
    print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))
print("the number of moves", m.objVal)

x[ 1, 1, 5, 1]=1.0
x[ 1, 2, 4, 1]=1.0
x[ 1, 3, 2, 2]=1.0
x[ 2, 1, 3, 1]=1.0
x[ 2, 2, 1, 3]=1.0
x[ 3, 1, 8, 1]=1.0
x[ 3, 2, 6, 1]=1.0
x[ 3, 3, 7, 1]=1.0
x[ 4, 1, 1, 1]=1.0
x[ 4, 1, 7, 4]=1.0
x[ 4, 2, 2, 1]=1.0
y[ 3, 4, 3]=1.0
y[ 4, 1, 1]=1.0
y[ 4, 2, 2]=1.0
the number of moves 3.0
